#### This notebook download stock prices from Yahoo Finance 

In [3]:
# df_adjOHLCV has data only for NYSE trading days, no weekend data
# df_adjOHLCV includes data for weekends when BTC trades
# read symbols in file to list syms_in_file
# download OHLCV data for symbols in syms_in_file
# drop symbols with all NaN in OHLCV columns from df
# rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
# drop weekend data by reindex to date index of index_symbol
# pickled df_adjOHLCV
# pickled df_adjOHLCV
# pickled symbols_df_adjOHLCV
# create df_symbols_close, sort df by symbols
# pickled df_symbols_close

In [4]:
import os


# Get the user's home directory Documents folder
path_dir = os.path.join(os.path.expanduser('~'), 'Documents/')
path_data_dump = path_dir + "VSCode_dump/"

print(f'path_dir: {path_dir}')
print(f'path_data_dump: {path_data_dump}')

path_dir: C:\Users\ping\Documents/
path_data_dump: C:\Users\ping\Documents/VSCode_dump/


In [5]:
import os
import sys


current_dir = os.getcwd()

# Parent directory where myUtils is located
path_utils = os.path.dirname(current_dir)
sys.path.append(path_utils)

In [39]:
# https://stackoverflow.com/questions/67690778/how-to-detect-failed-downloads-using-yfinance
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj, adjust_OHLC

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

verbose = False  # True prints more output

# #####################################
# # path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
# path_dir = "G:/My Drive/stocks/yfinance/"

# ##########################################
# path_data_dump = path_dir + "VSCode_dump/"
# # path_data_dump = path_dir + "VSCode_dump_trash/"


# filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'vg_symbols_4chars_max_clean.csv'  # symbols text file
filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded'  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_symbols_err = 'symbols_err.txt'  # file cumulates symbols with download errors

In [7]:
# # function to adjust OHLCV
# # https://github.com/ranaroussi/yfinance/issues/687
# def auto_adjust(data):
#     df = data.copy()
#     ratio = df["Close"] / df["Adj Close"]
#     df["Adj Open"] = df["Open"] / ratio
#     df["Adj High"] = df["High"] / ratio
#     df["Adj Low"] = df["Low"] / ratio

#     df.drop(
#         ["Open", "High", "Low", "Close"],
#         axis=1, inplace=True)

#     df.rename(columns={
#         "Adj Open": "Open", "Adj High": "High",
#         "Adj Low": "Low", "Adj Close": "Close"
#     }, inplace=True)

#     # df = df[["Open", "High", "Low", "Close", "Volume"]]
#     return df[["Open", "High", "Low", "Close", "Volume"]]

def close_vs_Yahoo (symbols, df):
  for symbol in symbols:
    s = df.iloc[-250]
    
##############################################
    # sDate = s.name.strftime('%Y-%m-%d')
    sDate = s.name
    
    sClose = s[symbol].Close

    df_sym = yf.Ticker(symbol).history(period='2y')
    # 10:45PM got Adj Close
    yhClose = df_sym.loc[sDate]['Close']

    abs_pct_diff = abs(1 - sClose/yhClose)*100
    print(f'symbol:  {symbol:>4}   Date: {sDate:13}df_Close: {sClose:>10.5f} \
    Yahoo_Close: {yhClose:>10.5f}   %_dif_Close: {abs_pct_diff:>7.5f}')
    if abs_pct_diff > .0001:
      msg_err = f'{symbol}  %_dif_Close > .0001'
      # raise SystemExit(msg_err)
      print(msg_err)  # even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
    if symbol == symbols[-1]:
      msg_done = f"No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close "
      print(msg_done)


In [8]:
# Stop if last date in df_adjOHLCV is the same as Yahoo download
index_symbol = "XOM"  

########################################
# df_XOM = yf.download(index_symbol)
# df_XOM = yf.download(index_symbol, period="10y")
df_XOM = yf_download_AdjOHLCV_noAutoAdj(index_symbol, period="10y")

# Change the DatetimeIndex to "yyyy-mm-dd" format
df_XOM.index = df_XOM.index.strftime('%Y-%m-%d')

# download_last_date = df_XOM.index[-1].strftime('%Y-%m-%d')
download_last_date = df_XOM.index[-1]

print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
print(f'download_last_date: {download_last_date}')

df = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

# # Change the DatetimeIndex to "yyyy-mm-dd" format
# df.index = df.index.strftime('%Y-%m-%d')

df_adjOHLCV_last_date = df.index[-1]
print(f'df_adjOHLCV_last_date: {df_adjOHLCV_last_date}')

# #####  SystemExit if data have been downloaded  #####
# if download_last_date == df_adjOHLCV_last_date:
#   msg_stop = f"df_adjOHLCV is up-to-date,\nlast date from df_adjOHLCV is {df_adjOHLCV_last_date},\nlast date from Yahoo's {index_symbol} download is {download_last_date}"
#   raise SystemExit(msg_stop )
# #####################################################

[*********************100%***********************]  1 of 1 completed


Full path to pickled df_adjOHLCV:  C:\Users\ping\Documents/VSCode_dump/df_adjOHLCV
download_last_date: 2024-09-04
df_adjOHLCV_last_date: 2024-09-03


In [9]:
# verify df test_symbols' close against Yahoo
# even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
test_symbols = ['A', 'LMT', 'SPY']
_df_list=[]
_df = yf_download_AdjOHLCV_noAutoAdj(test_symbols, verbose=False)

# Change the DatetimeIndex to "yyyy-mm-dd" format
_df.index = _df.index.strftime('%Y-%m-%d')

_df_list.append(_df)
_df = pd.concat(_df_list, axis=1)
# get unique symbols from column name
_l_symbols = list(set(_df.columns.get_level_values(0)))
_l_symbols.sort()

_df_adj_list=[]
# adjust OHLC using 'Adj Close'
for symbol in _l_symbols:
  _df1 = adjust_OHLC(_df[symbol])
  _df_adj_list.append(_df1)

_df_adj = pd.concat(_df_adj_list, axis=1)
_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
# create multilevel column names
_col_names = pd.MultiIndex.from_product([_l_symbols, _cols])
_df_adj.columns = _col_names

close_vs_Yahoo(test_symbols, _df_adj)

[*********************100%***********************]  3 of 3 completed


symbol:     A   Date: 2023-09-07   df_Close:  115.42808     Yahoo_Close:  115.42808   %_dif_Close: 0.00000
symbol:   LMT   Date: 2023-09-07   df_Close:  414.72043     Yahoo_Close:  414.72043   %_dif_Close: 0.00000
symbol:   SPY   Date: 2023-09-07   df_Close:  438.71356     Yahoo_Close:  438.71353   %_dif_Close: 0.00001
No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [10]:
# read symbols in file to a list and convert to chunks
#  each chunk must have more than 1 symbol, otherwise, symbol won't appear as column name
symbols_in_file = read_symbols_file(filename_symbols)
print(f'symbols in file: {len(symbols_in_file)}')
symbols_add = ['SPY', 'VGT', 'FTEC', 'QQQ', 'VWO', 'VTV', 'BNDX', 'USO', 'JJC', 'BCI', 'NG', 'GBTC', 'BTC-USD', 'ETH-USD', 'SGOV', 'GLD', 'MSTR']
print(f'symbols added:   {len(symbols_add)}')
symbols = symbols_in_file + symbols_add
# https://stackoverflow.com/questions/12897374/get-unique-values-from-a-list-in-python
symbols = list(set(symbols))  # keep unique symbols
symbols.sort()  # sort unique symbols in alphabetical order
print(f'unique symbols:  {len(symbols)}')
# symbols_chunks = chunked_list(symbols, 400)  # e.g. [['A', 'BB', ...], ['CC', 'DD', ...], ..., ['Z', 'ZWS', ...]]
symbols_chunks = chunked_list(symbols, 400)  # e.g. [['A', 'BB', ...], ['CC', 'DD', ...], ..., ['Z', 'ZWS', ...]]
print(f'number of chucks: {len(symbols_chunks)}')
for chunk in symbols_chunks:
  if len(chunk) == 1:
    raise SystemExit(f'ERROR, Must be more than 1 symbol in each chunk\nsymbols in chunk: {chunk}')

symbols in file: 1593
symbols added:   17
unique symbols:  1602
number of chucks: 5


In [11]:
df_list=[]
symbols_download_err = []
# took 24 minutes to download 1917 symbols without error caused by Yahoo
# for i, symbols in enumerate(symbols_chunks):
for i, symbols in enumerate(symbols_chunks):
  df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)

#####################################################  
  # Change the DatetimeIndex to "yyyy-mm-dd" format
  df.index = df.index.strftime('%Y-%m-%d')  

  symbols_download_err.append(list(shared._ERRORS.keys()))
  # print(f'symbols_download_err: {symbols_download_err}')
  print(f'\nsymbols_download_err: {symbols_download_err}')  
  df_list.append(df)
  # pause between download
  if i < len(symbols_chunks) - 1 :  # skip pause after last download
    print(f'downloaded symbols from chuck {i}, sleep start')
    # sleep in sec
    time.sleep(60)
    print(f'downloaded symbols from chuck {i}, sleep ends')
  else:
    print(f'downloaded symbols from all chucks')

print(f'symbols_download_err: {symbols_download_err}')

[*********************100%***********************]  400 of 400 completed

21 Failed downloads:
['CNHI', 'CDAY', 'BVH', 'AAWW', 'BKI', 'ATVI', 'ATCO', 'CPE', 'ABC', 'AIRC', 'ABB', 'DCP', 'AJRD']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (period=10y) (Yahoo error = "No data found, symbol may be delisted")')
['BRBR', 'CHK', 'ALGM', 'ABNB', 'CEG', 'CARR', 'CNXC']: YFInvalidPeriodError("%ticker%: Period '10y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', 'ytd', 'max']")
['CR']: YFInvalidPeriodError("%ticker%: Period '10y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', 'ytd', 'max']")



symbols_download_err: [['CNHI', 'BRBR', 'CDAY', 'CHK', 'BVH', 'AAWW', 'BKI', 'ALGM', 'ABNB', 'ATVI', 'ATCO', 'CEG', 'CPE', 'ABC', 'CR', 'AIRC', 'ABB', 'CARR', 'CNXC', 'DCP', 'AJRD']]
downloaded symbols from chuck 0, sleep start
downloaded symbols from chuck 0, sleep ends


[*********************100%***********************]  400 of 400 completed

22 Failed downloads:
['KDNY', 'ISEE', 'FLT', 'ERF', 'HHC', 'HEP', 'EVBG', 'IAA', 'DOOR', 'ICPT', 'FRC', 'DISH']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (period=10y) (Yahoo error = "No data found, symbol may be delisted")')
['ESAB', 'DTM', 'GFS', 'GXO', 'FYBR', 'DDOG', 'KD']: YFInvalidPeriodError("%ticker%: Period '10y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', 'ytd', 'max']")
['HZNP']: YFInvalidPeriodError("%ticker%: Period '10y' is invalid, must be one of ['1d', '5d']")
['GEHC', 'FISV']: YFInvalidPeriodError("%ticker%: Period '10y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', 'ytd', 'max']")



symbols_download_err: [['CNHI', 'BRBR', 'CDAY', 'CHK', 'BVH', 'AAWW', 'BKI', 'ALGM', 'ABNB', 'ATVI', 'ATCO', 'CEG', 'CPE', 'ABC', 'CR', 'AIRC', 'ABB', 'CARR', 'CNXC', 'DCP', 'AJRD'], ['KDNY', 'ESAB', 'DTM', 'ISEE', 'FLT', 'GFS', 'GXO', 'ERF', 'HHC', 'HZNP', 'GEHC', 'FYBR', 'HEP', 'DDOG', 'EVBG', 'FISV', 'IAA', 'DOOR', 'ICPT', 'KD', 'FRC', 'DISH']]
downloaded symbols from chuck 1, sleep start
downloaded symbols from chuck 1, sleep ends


[*********************100%***********************]  400 of 400 completed

25 Failed downloads:
['RIVN', 'OTIS', 'MP', 'PGNY', 'OGN', 'LCID', 'NARI', 'NVST']: YFInvalidPeriodError("%ticker%: Period '10y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', 'ytd', 'max']")
['LSI', 'NATI', 'PACW', 'NCR', 'PEAK', 'MDC', 'MGI', 'MODN', 'NUVA', 'PKI', 'NEX', 'RE', 'PXD', 'OFC', 'MMP']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (period=10y) (Yahoo error = "No data found, symbol may be delisted")')
['PDCE', 'OSTK']: YFInvalidPeriodError("%ticker%: Period '10y' is invalid, must be one of ['1d', '5d']")



symbols_download_err: [['CNHI', 'BRBR', 'CDAY', 'CHK', 'BVH', 'AAWW', 'BKI', 'ALGM', 'ABNB', 'ATVI', 'ATCO', 'CEG', 'CPE', 'ABC', 'CR', 'AIRC', 'ABB', 'CARR', 'CNXC', 'DCP', 'AJRD'], ['KDNY', 'ESAB', 'DTM', 'ISEE', 'FLT', 'GFS', 'GXO', 'ERF', 'HHC', 'HZNP', 'GEHC', 'FYBR', 'HEP', 'DDOG', 'EVBG', 'FISV', 'IAA', 'DOOR', 'ICPT', 'KD', 'FRC', 'DISH'], ['RIVN', 'LSI', 'NATI', 'PACW', 'NCR', 'PEAK', 'MDC', 'MGI', 'MODN', 'PDCE', 'NUVA', 'OSTK', 'OTIS', 'MP', 'PGNY', 'PKI', 'OGN', 'LCID', 'NEX', 'RE', 'PXD', 'OFC', 'MMP', 'NARI', 'NVST']]
downloaded symbols from chuck 2, sleep start
downloaded symbols from chuck 2, sleep ends


[*********************100%***********************]  400 of 400 completed

24 Failed downloads:
['VSCO', 'VNT', 'SITM', 'SHC', 'SGOV']: YFInvalidPeriodError("%ticker%: Period '10y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', 'ytd', 'max']")
['TA', 'SYNH', 'SIX', 'SGEN', 'TGH', 'TWNK', 'WWE', 'UNVR', 'SPLK', 'SRC', 'ROCC', 'TNP', 'SEAS', 'WIRE', 'SWAV', 'SJR', 'SCHN', 'VMW']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (period=10y) (Yahoo error = "No data found, symbol may be delisted")')
['SBNY']: YFInvalidPeriodError("%ticker%: Period '10y' is invalid, must be one of ['1d', '5d', '1mo', 'ytd', 'max']")



symbols_download_err: [['CNHI', 'BRBR', 'CDAY', 'CHK', 'BVH', 'AAWW', 'BKI', 'ALGM', 'ABNB', 'ATVI', 'ATCO', 'CEG', 'CPE', 'ABC', 'CR', 'AIRC', 'ABB', 'CARR', 'CNXC', 'DCP', 'AJRD'], ['KDNY', 'ESAB', 'DTM', 'ISEE', 'FLT', 'GFS', 'GXO', 'ERF', 'HHC', 'HZNP', 'GEHC', 'FYBR', 'HEP', 'DDOG', 'EVBG', 'FISV', 'IAA', 'DOOR', 'ICPT', 'KD', 'FRC', 'DISH'], ['RIVN', 'LSI', 'NATI', 'PACW', 'NCR', 'PEAK', 'MDC', 'MGI', 'MODN', 'PDCE', 'NUVA', 'OSTK', 'OTIS', 'MP', 'PGNY', 'PKI', 'OGN', 'LCID', 'NEX', 'RE', 'PXD', 'OFC', 'MMP', 'NARI', 'NVST'], ['VSCO', 'VNT', 'TA', 'SYNH', 'SBNY', 'SIX', 'SGEN', 'SITM', 'TGH', 'TWNK', 'WWE', 'UNVR', 'SPLK', 'SRC', 'ROCC', 'SHC', 'TNP', 'SEAS', 'SGOV', 'WIRE', 'SWAV', 'SJR', 'SCHN', 'VMW']]
downloaded symbols from chuck 3, sleep start
downloaded symbols from chuck 3, sleep ends


[*********************100%***********************]  2 of 2 completed


symbols_download_err: [['CNHI', 'BRBR', 'CDAY', 'CHK', 'BVH', 'AAWW', 'BKI', 'ALGM', 'ABNB', 'ATVI', 'ATCO', 'CEG', 'CPE', 'ABC', 'CR', 'AIRC', 'ABB', 'CARR', 'CNXC', 'DCP', 'AJRD'], ['KDNY', 'ESAB', 'DTM', 'ISEE', 'FLT', 'GFS', 'GXO', 'ERF', 'HHC', 'HZNP', 'GEHC', 'FYBR', 'HEP', 'DDOG', 'EVBG', 'FISV', 'IAA', 'DOOR', 'ICPT', 'KD', 'FRC', 'DISH'], ['RIVN', 'LSI', 'NATI', 'PACW', 'NCR', 'PEAK', 'MDC', 'MGI', 'MODN', 'PDCE', 'NUVA', 'OSTK', 'OTIS', 'MP', 'PGNY', 'PKI', 'OGN', 'LCID', 'NEX', 'RE', 'PXD', 'OFC', 'MMP', 'NARI', 'NVST'], ['VSCO', 'VNT', 'TA', 'SYNH', 'SBNY', 'SIX', 'SGEN', 'SITM', 'TGH', 'TWNK', 'WWE', 'UNVR', 'SPLK', 'SRC', 'ROCC', 'SHC', 'TNP', 'SEAS', 'SGOV', 'WIRE', 'SWAV', 'SJR', 'SCHN', 'VMW'], []]
downloaded symbols from all chucks
symbols_download_err: [['CNHI', 'BRBR', 'CDAY', 'CHK', 'BVH', 'AAWW', 'BKI', 'ALGM', 'ABNB', 'ATVI', 'ATCO', 'CEG', 'CPE', 'ABC', 'CR', 'AIRC', 'ABB', 'CARR', 'CNXC', 'DCP', 'AJRD'], ['KDNY', 'ESAB', 'DTM', 'ISEE', 'FLT', 'GFS', 'GXO', 'ER

In [12]:
# flatten symbols_download_err which is a list-of-lists
l_symbols_err = [val for sublist in symbols_download_err for val in sublist]
pickle_dump(l_symbols_err, path_data_dump, 'l_symbols_err')
l_symbols_err

['CNHI',
 'BRBR',
 'CDAY',
 'CHK',
 'BVH',
 'AAWW',
 'BKI',
 'ALGM',
 'ABNB',
 'ATVI',
 'ATCO',
 'CEG',
 'CPE',
 'ABC',
 'CR',
 'AIRC',
 'ABB',
 'CARR',
 'CNXC',
 'DCP',
 'AJRD',
 'KDNY',
 'ESAB',
 'DTM',
 'ISEE',
 'FLT',
 'GFS',
 'GXO',
 'ERF',
 'HHC',
 'HZNP',
 'GEHC',
 'FYBR',
 'HEP',
 'DDOG',
 'EVBG',
 'FISV',
 'IAA',
 'DOOR',
 'ICPT',
 'KD',
 'FRC',
 'DISH',
 'RIVN',
 'LSI',
 'NATI',
 'PACW',
 'NCR',
 'PEAK',
 'MDC',
 'MGI',
 'MODN',
 'PDCE',
 'NUVA',
 'OSTK',
 'OTIS',
 'MP',
 'PGNY',
 'PKI',
 'OGN',
 'LCID',
 'NEX',
 'RE',
 'PXD',
 'OFC',
 'MMP',
 'NARI',
 'NVST',
 'VSCO',
 'VNT',
 'TA',
 'SYNH',
 'SBNY',
 'SIX',
 'SGEN',
 'SITM',
 'TGH',
 'TWNK',
 'WWE',
 'UNVR',
 'SPLK',
 'SRC',
 'ROCC',
 'SHC',
 'TNP',
 'SEAS',
 'SGOV',
 'WIRE',
 'SWAV',
 'SJR',
 'SCHN',
 'VMW']

In [13]:
# if l_symbols_err:  # re-download symbols with download error 
#   symbols_download_err = []
#   # took 24 minutes to download 1917 symbols without error caused by Yahoo
#   # for i, symbols in enumerate(symbols_chunks):
#   for i, symbols in enumerate(l_symbols_err):
    # df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)
    # # Change the DatetimeIndex to "yyyy-mm-dd" format
    # df.index = df.index.strftime('%Y-%m-%d')  
#     symbols_download_err.append(list(shared._ERRORS.keys()))
#     # print(f'symbols_download_err: {symbols_download_err}')
#     print(f'\nsymbols_download_err: {symbols_download_err}')  
#     df_list.append(df)
#     # pause between download
#     if i < len(symbols_chunks) - 1 :  # skip pause after last download
#       print(f'downloaded symbols from chuck {i}, sleep start')
#       # sleep 78(18m 25s)
#       time.sleep(78)
#       print(f'downloaded symbols from chuck {i}, sleep ends')
#     else:
#       print(f'downloaded symbols from all chucks')

#   print(f'symbols_download_err: {symbols_download_err}')

In [14]:
df = pd.concat(df_list, axis=1)
if l_symbols_err:  # if list is not empty, drop symbols with errors
  print(f'l_symbols_err is not empty: {l_symbols_err}')  
  df.drop(l_symbols_err, axis=1, level=0, inplace=True)  # drop symbols with errors

l_symbols = list(set(df.columns.get_level_values(0)))  # get unique symbols
l_symbols.sort()
df = df[l_symbols]  # sort columns by list
print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
pickle_dump(df, path_data_dump, filename_pickled_df_OHLCVA_downloaded)  # save OHLCA data

l_symbols_err is not empty: ['CNHI', 'BRBR', 'CDAY', 'CHK', 'BVH', 'AAWW', 'BKI', 'ALGM', 'ABNB', 'ATVI', 'ATCO', 'CEG', 'CPE', 'ABC', 'CR', 'AIRC', 'ABB', 'CARR', 'CNXC', 'DCP', 'AJRD', 'KDNY', 'ESAB', 'DTM', 'ISEE', 'FLT', 'GFS', 'GXO', 'ERF', 'HHC', 'HZNP', 'GEHC', 'FYBR', 'HEP', 'DDOG', 'EVBG', 'FISV', 'IAA', 'DOOR', 'ICPT', 'KD', 'FRC', 'DISH', 'RIVN', 'LSI', 'NATI', 'PACW', 'NCR', 'PEAK', 'MDC', 'MGI', 'MODN', 'PDCE', 'NUVA', 'OSTK', 'OTIS', 'MP', 'PGNY', 'PKI', 'OGN', 'LCID', 'NEX', 'RE', 'PXD', 'OFC', 'MMP', 'NARI', 'NVST', 'VSCO', 'VNT', 'TA', 'SYNH', 'SBNY', 'SIX', 'SGEN', 'SITM', 'TGH', 'TWNK', 'WWE', 'UNVR', 'SPLK', 'SRC', 'ROCC', 'SHC', 'TNP', 'SEAS', 'SGOV', 'WIRE', 'SWAV', 'SJR', 'SCHN', 'VMW']
Full path to pickled df_OHLCVA_downloaded:  C:\Users\ping\Documents/VSCode_dump/df_OHLCVA_downloaded


In [15]:
# adjust OHLC
df_adj_list=[]
for symbol in l_symbols:
  _df = adjust_OHLC(df[symbol])
  df_adj_list.append(_df)

df_adj = pd.concat(df_adj_list, axis=1)
cols = ['Open', 'High', 'Low', 'Close', 'Volume']
col_names = pd.MultiIndex.from_product([l_symbols, cols])  # create multilevel column names
df_adj.columns = col_names  # reindex columns

print(f"Full path to pickled df_adjOHLCV: {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adj, path_data_dump, filename_pickled_df_adjOHLCV)  # save adjusted OHLCV data

Full path to pickled df_adjOHLCV: C:\Users\ping\Documents/VSCode_dump/df_adjOHLCV


In [16]:
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV)

In [17]:
# drop symbols with all NaN in OHLCV columns from df
df_adjOHLCV, symbols_OHLCV, symbols_dropped = drop_symbols_all_NaN(df_adjOHLCV, verbose)
print(f'symbols with all NaN dropped from df_adjOHLCV: {symbols_dropped}')

Symbol's OHLCV are all NaN: []
symbols with all NaN dropped from df_adjOHLCV: []


In [18]:
# rename columns OHLCV *ONLY AFTER* dropping symbols with all NaN from df,
#   symbols with all NaN has an added AdjClose column and will cause errors  
#  rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
#  .remove_unused_levels() prevents ValueError
#   e.g ValueError: On level 1, code max (5) >= length of level (5). NOTE: this index is in an inconsistent state
# The error may be caused by removing symbols from the dataframe with all NaN in OHLCV columns
df_adjOHLCV.columns = df_adjOHLCV.columns.remove_unused_levels()
# # set_levels reorders df columns in alphabetical order, so the list of column names also needs to be in alphabetical order
# df_adjOHLCV.columns = df_adjOHLCV.columns.set_levels(['close', 'high', 'low', 'open', 'volume'], level=1)

In [19]:
# drop weekend data by re-indexing to date-index of index_symbol
myNaN = float('nan')
# use Exxon's date as proxy for NYSE trading dates
df_adjOHLCV = df_adjOHLCV.reindex(df_XOM.index, fill_value=myNaN)

In [20]:
df_adjOHLCV

A                                                        AA                                                    AAL                                                    AAON                                                    AAP                                                       AAPL                                                          AB                                                   ABBV                                                        ABM                                                   ABR                                                     ABT                                                       ACGL                                                       ACHC                                                   ACIW                                                   ACLS                                                       ACM                                                       ACN                                                 ACRS        \
                  Open        High         Low       Close     Volume       Open       High        Low      Close     Volume       Open       High        Low      Close      Volume       Open       High        Low      Close    Volume        Open        High         Low       Close     Volume        Open        High         Low       Close       Volume       Open       High        Low      Close    Volume        Open        High         Low       Close      Volume       Open       High        Low      Close    Volume       Open       High        Low      Close     Volume        Open        High         Low       Close     Volume        Open        High         Low       Close      Volume       Open       High        Low      Close     Volume       Open       High        Low      Close    Volume        Open        High         Low       Close    Volume        Open        High        Low       Close     Volume        Open        High         Low       Close     Volume  Open  High   
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
2014-09-05   37.880348   38.189547   37.696143   38.104023  1268405.0  38.841848  39.250472  38.614837  39.227772  3995755.0  36.396995  36.586022  35.640891  35.773209  12067600.0  11.702786  11.914206  11.572202  11.889334  207000.0  124.875247  125.363862  124.006620  124.649040   532800.0   21.931314   22.062279   21.822544   21.969049  233828000.0  11.644610  11.687532  11.515846  11.541598  149900.0   36.706361   37.004032   36.296233   37.004032   9914300.0  22.964932  23.326068  22.866442  23.186539  256900.0   2.655415   2.674601   2.651577   2.663089    53900.0   35.418122   35.484199   35.062950   35.475941  3484300.0   18.363333   18.396667   18.320000   18.333332   3841800.0  49.230000  50.360001  48.840000  50.299999   244200.0  19.320000  19.530001  19.190001  19.490000  278900.0    7.880000    8.000000    7.880000    7.960000   38725.0   36.071894   36.540360  35.905981   36.472042   970500.0   68.610565   68.921289   68.417418   68.719742  2276700.0   NaN   NaN   
2014-09-08   37.972454   38.130343   

In [21]:
# pickle df_adjOHLCV and symbols
# print(f"Full path to pickled df_adjOHLCV_downloaded:  {path_data_dump}{filename_pickled_df_adjOHLCV_downloaded}")
# pickle_dump(df_adjOHLCV_downloaded, path_data_dump, filename_pickled_df_adjOHLCV_downloaded, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adjOHLCV, path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
pickle_dump(symbols_OHLCV, path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_adjOHLCV:  C:\Users\ping\Documents/VSCode_dump/df_adjOHLCV
Full path to pickled symbols_df_adjOHLCV:  C:\Users\ping\Documents/VSCode_dump/symbols_df_adjOHLCV


In [22]:
symbols_OHLCV = list(set([i[0] for i in list(df_adjOHLCV)]))
df_symbols_close = df_adjOHLCV.xs('Close', level=1, axis=1)
pickle_dump(df_symbols_close, path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

In [23]:
# retrieve pickled files

print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
df_OHLCVA_downloaded = pickle_load(path_data_dump, filename_pickled_df_OHLCVA_downloaded, verbose=verbose)

print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
symbols_df = pickle_load(path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_OHLCVA_downloaded:  C:\Users\ping\Documents/VSCode_dump/df_OHLCVA_downloaded
Full path to pickled df_adjOHLCV:  C:\Users\ping\Documents/VSCode_dump/df_adjOHLCV
Full path to pickled df_symbols_close:  C:\Users\ping\Documents/VSCode_dump/df_symbols_close
Full path to pickled symbols_df_adjOHLCV:  C:\Users\ping\Documents/VSCode_dump/symbols_df_adjOHLCV


In [24]:
print(f'df_OHLCVA_downloaded.info():\n{df_OHLCVA_downloaded.info()}\n')
print(f'df_adjOHLCV.info():\n{df_adjOHLCV.info()}\n')
print(f'df_close.info():\n{df_close.info()}\n')
print(f'len(symbols_df):\n{len(symbols_df)}\n')

<class 'pandas.core.frame.DataFrame'>
Index: 3650 entries, 2014-09-05 to 2024-09-05
Columns: 9060 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(9060)
memory usage: 252.3+ MB
df_OHLCVA_downloaded.info():
None

<class 'pandas.core.frame.DataFrame'>
Index: 2516 entries, 2014-09-05 to 2024-09-04
Columns: 7550 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(7550)
memory usage: 144.9+ MB
df_adjOHLCV.info():
None

<class 'pandas.core.frame.DataFrame'>
Index: 2516 entries, 2014-09-05 to 2024-09-04
Columns: 1510 entries, A to ZWS
dtypes: float64(1510)
memory usage: 29.0+ MB
df_close.info():
None

len(symbols_df):
1510



In [25]:
count_symbols = len(symbols_df)
count_df_close_cols = len(df_close.columns)
count_cols_df_OHLCVA_downloaded = len(df_OHLCVA_downloaded.columns)
count_cols_df_adjOHLCV = len(df_adjOHLCV.columns)
if count_symbols != count_df_close_cols:
  raise SystemExit(f'symbol count: {count_symbols} does not equal df_close column count: {count_df_close_cols}')
if count_symbols * 6 != count_cols_df_OHLCVA_downloaded:
  raise SystemExit(f'6 x symbol count: {6 * count_symbols} does not equal df_OHLCVA_downloaded column count: {count_cols_df_OHLCVA_downloaded}')
if count_symbols * 5 != count_cols_df_adjOHLCV:
  raise SystemExit(f'5 x symbol count: {5 * count_symbols} does not equal df_adjOHLCV column count: {count_cols_df_adjOHLCV}')
print(f'No error found, Column counts in df_close, df_OHLCVA, df_adjOHLCV matched symbol count')


No error found, Column counts in df_close, df_OHLCVA, df_adjOHLCV matched symbol count


In [26]:
print(f'count_symbols: {count_symbols}')
print(f'count_df_close_cols: {count_df_close_cols}')
print(f'count_cols_df_OHLCVA_downloaded: {count_cols_df_OHLCVA_downloaded}')
print(f'count_cols_df_adjOHLCV: {count_cols_df_adjOHLCV}')

count_symbols: 1510
count_df_close_cols: 1510
count_cols_df_OHLCVA_downloaded: 9060
count_cols_df_adjOHLCV: 7550


In [27]:
close_vs_Yahoo(test_symbols, df_adjOHLCV)

symbol:     A   Date: 2023-09-07   df_Close:  115.42809     Yahoo_Close:  115.42808   %_dif_Close: 0.00001
symbol:   LMT   Date: 2023-09-07   df_Close:  414.72040     Yahoo_Close:  414.72040   %_dif_Close: 0.00000
symbol:   SPY   Date: 2023-09-07   df_Close:  438.71353     Yahoo_Close:  438.71356   %_dif_Close: 0.00001
No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [28]:
syms_def = ['NOC', 'HII', 'AVAV', 'LMT', 'CW', 'HEI']
_df = df_close[syms_def]
_df.tail()


,NOC,HII,AVAV,LMT,CW,HEI
Date,,,,,,
2024-08-28,511.537994,278.762909,193.860001,560.852844,312.160004,250.350006
2024-08-29,517.305176,281.559967,203.610001,563.706909,314.399994,255.020004
2024-08-30,521.150024,282.769989,203.759995,564.950012,315.859985,256.559998
2024-09-03,522.690002,275.450012,192.259995,567.219971,302.970001,247.740005
2024-09-04,523.679993,274.269989,194.039993,574.780029,302.820007,252.610001


In [ ]:
l_symbols_err

#### Below cells cumulates symbols with download errors and count how many times the error appears. If the count is high, that may be an indiction the symbol may be delisted, and consider removing it from 'my_symbols.csv'   

In [43]:
"""
This script counts the occurrences of each item in a file and sorts the results in descending order, first by count and then by item name.
"""

import collections

def count_item_occurrences(filename: str) -> list[tuple[str, int]]:
    """
    Counts the occurrences of each item in a file and returns a sorted list of tuples.

    Args:
        filename: The name of the file to process.

    Returns:
        A list of tuples, where each tuple contains an item and its corresponding count. The list is sorted in descending order first by count and then by item name.
    """

    item_counts = collections.Counter()
    with open(filename, "r") as f:
        for line in f:
            item_counts[line.strip()] += 1
    sorted_counts = sorted(item_counts.items(), key=lambda x: (-x[1], x[0]))
    return sorted_counts

In [45]:
# Append the symbol error list to the file
with open(filename_symbols_err, "a") as f:
    f.writelines(str(item) + "\n" for item in l_symbols_err)

# Count and sort item occurrences
sorted_counts = count_item_occurrences(filename_symbols_err)

# Print the results
for item, count in sorted_counts:
    print(f"{item}: {count}")

AAWW: 1
ABB: 1
ABC: 1
ABNB: 1
AIRC: 1
AJRD: 1
ALGM: 1
ATCO: 1
ATVI: 1
BKI: 1
BRBR: 1
BVH: 1
CARR: 1
CDAY: 1
CEG: 1
CHK: 1
CNHI: 1
CNXC: 1
CPE: 1
CR: 1
DCP: 1
DDOG: 1
DISH: 1
DOOR: 1
DTM: 1
ERF: 1
ESAB: 1
EVBG: 1
FISV: 1
FLT: 1
FRC: 1
FYBR: 1
GEHC: 1
GFS: 1
GXO: 1
HEP: 1
HHC: 1
HZNP: 1
IAA: 1
ICPT: 1
ISEE: 1
KD: 1
KDNY: 1
LCID: 1
LSI: 1
MDC: 1
MGI: 1
MMP: 1
MODN: 1
MP: 1
NARI: 1
NATI: 1
NCR: 1
NEX: 1
NUVA: 1
NVST: 1
OFC: 1
OGN: 1
OSTK: 1
OTIS: 1
PACW: 1
PDCE: 1
PEAK: 1
PGNY: 1
PKI: 1
PXD: 1
RE: 1
RIVN: 1
ROCC: 1
SBNY: 1
SCHN: 1
SEAS: 1
SGEN: 1
SGOV: 1
SHC: 1
SITM: 1
SIX: 1
SJR: 1
SPLK: 1
SRC: 1
SWAV: 1
SYNH: 1
TA: 1
TGH: 1
TNP: 1
TWNK: 1
UNVR: 1
VMW: 1
VNT: 1
VSCO: 1
WIRE: 1
WWE: 1
